In [4]:
import pandas as pd
import numpy as np
import cvxpy as cvx
import random
import time
from __future__ import print_function
from builtins import input


from sporco.admm import bpdn
### using the ADMM algorithm 
### we can also use the PGM algorithm 
from sporco import util
from sporco import plot
plot.config_notebook_plotting()

from scipy.linalg import hadamard

import matplotlib.pyplot as plt

from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold

import time 
from scipy import stats
from matplotlib import pyplot
iHiV = pd.read_pickle("~/bge-analysis-simv3/iHiV.pkl")
B = pd.read_pickle("~/bge-analysis-simv3/B.pkl")


import warnings
warnings.filterwarnings("ignore")

In [5]:
X = pd.read_pickle("~/bge-analysis-simv3/X16.pkl")

In [6]:
from sklearn.metrics import r2_score

In [7]:
labdl = np.round(np.logspace(-5,0.8,5),5)
lam = list(labdl)
lam.append([7.0,9.0,10.0,15.0,20.0])
lamf = list(pd.DataFrame(lam)[0].explode().values)
lamf.sort()

In [8]:
lamf

[1e-05, 0.00028, 0.00794, 0.22387, 6.30957, 7.0, 9.0, 10.0, 15.0, 20.0]

In [15]:
n = 2**16
random_state = None
laml = lamf

opt = bpdn.BPDN.Options({'Verbose': False, 'MaxMainIter': 500,
                         'RelStopTol': 1e-8, 'AutoRho': {'RsdlTarget': 1.0}})

sst_redv1 = pd.read_pickle("~/compressed_sensingv1/realdatasets/OV-16sp-FUNCTIONS/16-list1-allmetabols.pkl")


metabolite_list = ["butyrate"]

In [13]:
whper = [] 
linper = [] 
for cv_splits in [3,5]:
    for metabol in metabolite_list: 
        datab = sst_redv1[str(metabol)]
        databdf = pd.DataFrame(datab)
        datap = list(databdf.index)
        for runs in range(3):
                xs = list(datap)
                random.shuffle(xs)
                data_present = np.array(xs)
                kf = KFold(n_splits=cv_splits)
                kf.get_n_splits(data_present)
                KFold(n_splits=cv_splits, random_state=random_state, shuffle=True)
                for lmda in laml:
                    for train_index, test_index in kf.split(data_present):
                        X_train, X_test = data_present[train_index], data_present[test_index]
                        rinew = list(X_train)
                        rileft = list(X_test)
                        D = iHiV[rinew,:]
                        y2 = databdf.T[rinew].T.values
                        yact = databdf.T[rileft].T.values
                        s3 = y2 
                        b = bpdn.BPDN(D, s3, lmda, opt)
                        x = b.solve()
                        yrecon = np.dot(iHiV,x.ravel())
                        whper.append([runs,lmda,str(metabol),yrecon[rileft],yact,rileft,rinew,cv_splits,len(rileft),len(rinew)])


0 1e-05 3
0 1e-05 3
0 1e-05 3
0 0.00028 3
0 0.00028 3
0 0.00028 3
0 0.00794 3
0 0.00794 3
0 0.00794 3
0 0.22387 3
0 0.22387 3
0 0.22387 3
0 6.30957 3
0 6.30957 3
0 6.30957 3
0 7.0 3
0 7.0 3
0 7.0 3
0 9.0 3
0 9.0 3
0 9.0 3
0 10.0 3
0 10.0 3
0 10.0 3
0 15.0 3
0 15.0 3
0 15.0 3
0 20.0 3
0 20.0 3
0 20.0 3
1 1e-05 3
1 1e-05 3
1 1e-05 3
1 0.00028 3
1 0.00028 3
1 0.00028 3
1 0.00794 3
1 0.00794 3
1 0.00794 3
1 0.22387 3
1 0.22387 3
1 0.22387 3
1 6.30957 3
1 6.30957 3
1 6.30957 3
1 7.0 3
1 7.0 3
1 7.0 3
1 9.0 3
1 9.0 3
1 9.0 3
1 10.0 3
1 10.0 3
1 10.0 3
1 15.0 3
1 15.0 3
1 15.0 3
1 20.0 3
1 20.0 3
1 20.0 3
2 1e-05 3
2 1e-05 3
2 1e-05 3
2 0.00028 3
2 0.00028 3
2 0.00028 3
2 0.00794 3
2 0.00794 3
2 0.00794 3
2 0.22387 3
2 0.22387 3
2 0.22387 3
2 6.30957 3
2 6.30957 3
2 6.30957 3
2 7.0 3
2 7.0 3
2 7.0 3
2 9.0 3
2 9.0 3
2 9.0 3
2 10.0 3
2 10.0 3
2 10.0 3
2 15.0 3
2 15.0 3
2 15.0 3
2 20.0 3
2 20.0 3
2 20.0 3
0 1e-05 5
0 1e-05 5
0 1e-05 5
0 1e-05 5
0 1e-05 5
0 0.00028 5
0 0.00028 5
0 0.00028 5
0 0.0

In [14]:
whdf  = pd.DataFrame(whper)
lindf = pd.DataFrame(linper)
pd.to_pickle(whdf,"3n5_cv_WH"+str(metabol)+".pkl")
pd.to_pickle(lindf,"3n5_cv_l1"+str(metabol)+".pkl")

In [27]:
whdf  = pd.DataFrame(whper)
lindf = pd.DataFrame(linper)

In [2]:
import pandas as pd
pd.read_pickle("~/compressed_sensingv1/realdatasets/OV-16sp-FUNCTIONS-v3/3n5_cv_linmetabols.pkl")

,0,1,2,3,4,5,6,7,8,9
0,0,0.0000,butyrate,"[52.31209207735067, 49.7609735371106, 8.989004...","[[46.5489], [41.2675916], [13.037733333333335]...","[174, 45, 9, 2177, 173, 40967, 1, 165, 1440, 3...","[48, 211, 163, 547, 8, 52, 65, 57349, 1056, 16...",3,63,124
1,0,0.0000,butyrate,"[33.13199415472038, 18.0731336579248, 51.50889...","[[34.00846666666667], [23.956022349999998], [4...","[48, 211, 163, 547, 8, 52, 65, 57349, 1056, 16...","[174, 45, 9, 2177, 173, 40967, 1, 165, 1440, 3...",3,62,125
2,0,0.0000,butyrate,"[9.746205792887379, -8.238019914544836, 38.660...","[[7.6138101], [0.0], [15.37545], [12.847283333...","[4103, 32896, 1187, 6, 544, 5, 28675, 4096, 13...","[174, 45, 9, 2177, 173, 40967, 1, 165, 1440, 3...",3,62,125
3,0,0.0003,butyrate,"[51.838486675876, 56.30587014891403, 12.040246...","[[46.5489], [41.2675916], [13.037733333333335]...","[174, 45, 9, 2177, 173, 40967, 1, 165, 1440, 3...","[48, 211, 163, 547, 8, 52, 65, 57349, 1056, 16...",3,63,124
4,0,0.0003,butyrate,"[21.899596212278407, 13.612856718664716, 42.85...","[[34.00846666666667], [23.956022349999998], [4...","[48, 211, 163, 547, 8, 52, 65, 57349, 1056, 16...","[174, 45, 9, 2177, 173, 40967, 1, 165, 1440, 3...",3,62,125
...,...,...,...,...,...,...,...,...,...,...
1171,4,6.3096,acetate,"[19.27100411297496, 12.066800828414927, 10.965...","[[20.86985], [12.949066666666667], [10.3707400...","[45188, 8224, 37, 4128, 32, 2182, 8, 7, 8355, ...","[24707, 8231, 18, 4224, 8194, 2084, 16929, 410...",5,38,149
1172,4,6.3096,acetate,"[11.811107093995759, 4.919991411476303, 15.882...","[[12.8946], [3.957883333333333], [17.7787], [4...","[1188, 36, 25121, 512, 16, 8833, 4, 514, 242, ...","[24707, 8231, 18, 4224, 8194, 2084, 16929, 410...",5,37,150
1173,4,6.3096,acetate,"[10.066121041236322, 15.22182644158697, 8.6369...","[[8.32125], [29.748649999999998], [15.36565], ...","[37024, 66, 39, 131, 41093, 32768, 2094, 289, ...","[24707, 8231, 18, 4224, 8194, 2084, 16929, 410...",5,37,150
1174,4,6.3096,acetate,"[32.9358158210388, 8.353530545285693, 27.91382...","[[10.011433333333335], [2.9793], [17.727375000...","[33028, 160, 1024, 128, 32770, 165, 2257, 288,...","[24707, 8231, 18, 4224, 8194, 2084, 16929, 410...",5,37,150
